In [4]:
from random import choice
from string import ascii_lowercase

def generate_random_string(length=10):
    letters = ascii_lowercase
    return ''.join(choice(letters) for i in range(length))

In [8]:
s3_bucket = "apper-training-sagemaker-deployments"
prefix = generate_random_string()

In [12]:
print(prefix)

wnrnepkqmn


In [9]:
model_data = f"s3://{s3_bucket}/{prefix}/files/model.knn.tar.gz"

In [11]:
!aws s3 cp files/model.knn.tar.gz {model_data}

upload: files/model.knn.tar.gz to s3://apper-training-sagemaker-deployments/wnrnepkqmn/files/model.knn.tar.gz


In [13]:
from sagemaker import Model
from sagemaker import KNNPredictor
from sagemaker.image_uris import retrieve

In [14]:
image_uri = retrieve("knn", region="us-east-1")
image_uri

'382416733822.dkr.ecr.us-east-1.amazonaws.com/knn:1'

In [15]:
import sagemaker
from sagemaker import get_execution_role

session = sagemaker.Session()
role = get_execution_role()

model = Model(model_data=model_data,
              image_uri=image_uri,
              role=role,
              predictor_cls=KNNPredictor)

In [16]:
model.__dict__

{'model_data': 's3://apper-training-sagemaker-deployments/wnrnepkqmn/files/model.knn.tar.gz',
 'image_uri': '382416733822.dkr.ecr.us-east-1.amazonaws.com/knn:1',
 'role': 'arn:aws:iam::305262579855:role/service-role/AmazonSageMaker-ExecutionRole-20210507T101941',
 'predictor_cls': sagemaker.amazon.knn.KNNPredictor,
 'env': {},
 'name': None,
 '_base_name': None,
 'vpc_config': None,
 'sagemaker_session': None,
 'endpoint_name': None,
 '_is_compiled_model': False,
 '_compilation_job_name': None,
 '_is_edge_packaged_model': False,
 '_enable_network_isolation': False,
 'model_kms_key': None,
 'image_config': None}

In [17]:
predictor = model.deploy(initial_instance_count=1,
                         instance_type='ml.t2.medium')

--------------------------!

In [18]:
import numpy as np

test_data = np.array([0.5, 0.5, 0.5, 0.9, 0.9])
predictor.predict(test_data)

[label {
   key: "predicted_label"
   value {
     float64_tensor {
       values: 1.0
     }
   }
 }]

In [19]:
predictor.delete_endpoint()